In [1]:

import os
#chat = open("../_chat.txt", encoding="utf-8")
with open("../_chat.txt",  encoding="utf-8") as f:
    chat = [line.strip('\n') for line in f]

## Clean Data

In [2]:
import re
joinedChat=' '.join(chat)
print(joinedChat[:500])

emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
joinedChat=(emoj.sub(r'', joinedChat)) # no emoji

joinedChat=re.sub(r"\[[^\]]*\]", '\n', joinedChat)
joinedChat=re.sub(r" Elif Yavrum:", 'Elif Yavrum:', joinedChat)
joinedChat=re.sub(r" Dogu Tamgac:", 'Dogu Tamgac:', joinedChat)
joinedChat=joinedChat.replace("\u200e", "")
print(joinedChat[:500])

[04.10.17, 17:32:35] Elif Yavrum: ‎Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. [04.10.17, 17:32:35] Elif Yavrum: Numaram degisti ekle beni bro [04.10.17, 18:14:23] Dogu Tamgac: Ekledim bro [05.10.17, 12:14:44] Elif Yavrum: Kk [11.10.17, 21:24:19] Dogu Tamgac: Telefon aldim ha [11.10.17, 21:24:21] Dogu Tamgac: Ekle [11.10.17, 22:04:03] Elif Yavrum: Tamam eklerim ha [11.10.17, 22:12:36] Dogu Tamgac: Napan [11.10.17, 22:13

Elif Yavrum: Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. 
Elif Yavrum: Numaram degisti ekle beni bro 
Dogu Tamgac: Ekledim bro 
Elif Yavrum: Kk 
Dogu Tamgac: Telefon aldim ha 
Dogu Tamgac: Ekle 
Elif Yavrum: Tamam eklerim ha 
Dogu Tamgac: Napan 
Elif Yavrum: Selinlerle viski iciyoruz 
Elif Yavrum: Sen 
Dogu Tamgac: Ben de kayralardayim 
Dogu Tamgac: Takiliyoz 
Dogu Tamgac: Dersler basladi mi 
Elif Yavrum: Yok 
Elif Ya

In [3]:
chars=sorted(list(set(joinedChat)))

In [4]:
vocab_size=len(set(chars))
print(''.join(chars))
print(vocab_size)

	
 !"#%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ]^_`abcdefghijklmnopqrstuvwxyz|}~ £¨«®¯´·»ÇÖ×ÜßâäçéêöüğİıŞş̈‐—’“”„•…€₺⃣
131


# Building character level language model. <br>
stoi: Map strings to integers. <br>
Then apply it and create encoding and decoding lambda functions <br>
(Normally you can use sentencepiece from google or tiktoken)

In [5]:
stoi={ch:i for i,ch in enumerate(chars)}
itoi={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [stoi[i] for i in s]
decode=lambda l: ''.join([itoi[i] for i in l])
print(encode("l l"))
print(decode(encode("l l")))

[75, 2, 75]
l l


In [6]:
import torch
torch.cuda.is_available()
data=torch.tensor(encode(joinedChat), dtype=torch.long)
print(data.shape , data.dtype)
print(data[:100])

torch.Size([3742194]) torch.int64
tensor([ 1, 38, 75, 72, 69,  2, 58, 64, 85, 81, 84, 76, 27,  2, 46, 68, 82, 82,
        64, 70, 68, 82,  2, 64, 77, 67,  2, 66, 64, 75, 75, 82,  2, 64, 81, 68,
         2, 68, 77, 67, 14, 83, 78, 14, 68, 77, 67,  2, 68, 77, 66, 81, 88, 79,
        83, 68, 67, 15,  2, 47, 78,  2, 78, 77, 68,  2, 78, 84, 83, 82, 72, 67,
        68,  2, 78, 69,  2, 83, 71, 72, 82,  2, 66, 71, 64, 83, 13,  2, 77, 78,
        83,  2, 68, 85, 68, 77,  2, 56, 71, 64])


In [7]:
n = int(0.95 * len(data))
train_data=data[:n]
val_data=data[n:]

Now take the data asa blocks. We chose the block size 8 and initialized the length of the training data with 9 as there are 8 positions to guess

In [8]:
block_size= 8
batch_size= 4

train_data[:block_size+1]

tensor([ 1, 38, 75, 72, 69,  2, 58, 64, 85])

In [9]:
torch.manual_seed(1337) # to be able to reproduce
def get_batch(split):
    data=train_data if split=='train' else val_data
    #shape batchsize row vector
    ix=torch.randint(len(data) - block_size, (batch_size,))
    #stacked multiple sentences as row vectors
    x=torch.stack([data[i:i+block_size] for i in ix])
    #to train
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y
xb,yb= get_batch('train')
print("input:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)



input:
torch.Size([4, 8])
tensor([[38, 75, 72, 69,  2, 58, 64, 85],
        [58, 64, 85, 81, 84, 76, 27,  2],
        [81, 67, 64, 81, 75, 64, 88, 67],
        [74, 82, 64,  2,  1, 38, 75, 72]])
targets:
torch.Size([4, 8])
tensor([[75, 72, 69,  2, 58, 64, 85, 81],
        [64, 85, 81, 84, 76, 27,  2, 34],
        [67, 64, 81, 75, 64, 88, 67, 72],
        [82, 64,  2,  1, 38, 75, 72, 69]])


# Using Bigram model=> Simple fast easy to understand
Predict the next word from the earlier word.(In our case character) Conditional probability

In [34]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

vocab_size=len(set(chars))
print(vocab_size)
#get the earlier character then propagate it 

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab):
        super().__init__()
        #wrapper essentialy a table by number of characters*number of characters where you have the scores
        self.token_embedding_table=nn.Embedding(vocab, vocab)
    
    def forward(self, idx, targets=None):
        logits=self.token_embedding_table(idx)
        #logits B t C where B=4 batchsize, T=8 chunks, C=number of characters
       
        if targets is None:
            loss=None
        else:
            B,T,C=logits.shape
            print(logits.shape)
            print(logits)
            #remove Chunk indexes like appending together
            logits= logits.view(B*T, C)
            print(logits.shape)
            print(logits)
            targets= targets.view(B*T)
            loss=F.cross_entropy(logits, targets)
        return logits, loss # scores
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            #Predictions
            logits, loss=self(idx)
            #Only take the last characters from chunks
            logits=logits[:,-1,:]
            print(logits)
            probs=F.softmax(logits, dim=-1)
            #get one sample from the softmax
            idx_next=torch.multinomial(probs, num_samples=1) # (B,1)
            #append the word to the chunk
            idx=torch.cat((idx, idx_next),dim=1)  # (B,T+1)
        return idx
            
m=BigramLanguageModel(vocab_size)
logits, loss=m(xb,yb) # forward
print(logits.shape)
# estimated loss ln(1/131)
print(loss)

print(decode(m.generate(torch.zeros((1,1),dtype=torch.long), max_new_tokens=100)[0].tolist()))

131
torch.Size([4, 8, 131])
tensor([[[ 3.2046, -0.8989,  0.8825,  ...,  1.2447,  0.9085, -0.1448],
         [ 1.2984, -0.6068,  0.9640,  ..., -0.7720, -0.3192, -2.0183],
         [ 1.1218, -0.3172, -1.5262,  ..., -0.9420,  0.7977, -1.1569],
         ...,
         [-0.6899, -0.9970, -0.0979,  ...,  1.5133,  0.0898,  0.3081],
         [ 1.3283, -0.5417, -1.8746,  ...,  0.4242, -0.2671,  1.1967],
         [ 0.0876, -0.2943, -0.4384,  ..., -0.9477,  1.7198,  0.5248]],

        [[-0.6899, -0.9970, -0.0979,  ...,  1.5133,  0.0898,  0.3081],
         [ 1.3283, -0.5417, -1.8746,  ...,  0.4242, -0.2671,  1.1967],
         [ 0.0876, -0.2943, -0.4384,  ..., -0.9477,  1.7198,  0.5248],
         ...,
         [-1.0443, -0.7578,  2.2483,  ..., -1.4310, -0.1728,  0.9693],
         [-0.2240, -1.0157, -0.1161,  ..., -0.3234, -0.9876, -0.1603],
         [ 1.4237,  0.2617, -1.8528,  ...,  0.4160,  0.3362, -0.4512]],

        [[-1.8985, -0.4794, -0.0575,  ..., -1.3935, -0.2010,  1.3311],
         [ 0.4177

import ntlk
AI_tokens=word_tokenize(joinedChat)
print(AI_tokens[:500])

from nltk.probability import FreqDist
fdist=FreqDist()
for word in AI_tokens:
    fdist[word.lower()]+=1
fdist